In [1]:
import sys
sys.path.append('./paralel_planarity_criterion')

import numpy as np
import random, time

import paralel_methods as pm
from SAT2_solver import SAT2_solver

In [2]:
# 🎲 Generate a random seed
seed = random.randint(0, 2**32 - 1)
print("Using seed:", seed)

# 🌱 Seed NumPy’s RNG
np.random.seed(seed)

Using seed: 2545199465


In [5]:
N = 250 
A = np.random.randint(0, 2, size=(N, N)).astype(np.int8)
B = A.T.copy() 

start_cpu = time.perf_counter()
C_paralel = pm.multiply_or_and(A, B)
end_cpu = time.perf_counter()

print("\nMatriz A:")
print(A)
print("\nMatriz B:")
print(B)
print()
print("----------------------------")
print("Paralel GPU implementation")
print(f"GPU OR-AND Time: {(end_cpu - start_cpu)*1000:.2f} ms")
print("\nResultado C (OR-AND):")
print(C_paralel)


# --- CPU: Pure Python ---
solver = SAT2_solver()

start_cpu = time.time()
C_sequential = solver.mult_matrix_or_and(A.tolist(), B.tolist())  # Convert np arrays to lists
end_cpu = time.time()

print()
print("--------------------------")
print("Sequential CPU implementation")
print(f"CPU OR-AND Time: {(end_cpu - start_cpu)*1000:.2f} ms")
print("\nResultado C (OR-AND):")
print(np.array(C_sequential, dtype=np.int8))


Matriz A:
[[1 1 1 ... 1 1 0]
 [1 0 0 ... 1 0 0]
 [0 1 1 ... 1 0 1]
 ...
 [1 0 1 ... 1 0 0]
 [0 1 1 ... 1 0 0]
 [0 1 1 ... 1 1 0]]

Matriz B:
[[1 1 0 ... 1 0 0]
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]]

----------------------------
Paralel GPU implementation
GPU OR-AND Time: 209.06 ms

Resultado C (OR-AND):
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]

--------------------------
Sequential CPU implementation
CPU OR-AND Time: 1832.10 ms

Resultado C (OR-AND):
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [4]:
import numpy as np
import time
from SAT2_solver import SAT2_solver
import pandas as pd

solver = SAT2_solver()

# Parámetros
sizes = list(range(10, 251, 20))  # Tamaños de matriz desde 10 hasta 250 en pasos de 20
repeticiones = 5  # Número de repeticiones por tamaño

# Resultados acumulados
resultados = []

for N in sizes:
    tiempos_gpu = []
    tiempos_cpu = []
    
    for _ in range(repeticiones):
        A = np.random.randint(0, 2, size=(N, N)).astype(np.int8)
        B = A.T.copy()

        # GPU Paralelo 
        start_gpu = time.perf_counter()
        _ = pm.multiply_or_and(A, B)
        end_gpu = time.perf_counter()
        tiempos_gpu.append((end_gpu - start_gpu) * 1000)

        # CPU Secuencial
        start_cpu = time.perf_counter()
        _ = solver.mult_matrix_or_and(A.tolist(), B.tolist())
        end_cpu = time.perf_counter()
        tiempos_cpu.append((end_cpu - start_cpu) * 1000)

    resultados.append({
        "Tamaño de Matriz": N,
        "Tiempo Promedio GPU (ms)": np.mean(tiempos_gpu),
        "Tiempo Promedio CPU (ms)": np.mean(tiempos_cpu)
    })

# Mostrar resultados como tabla
df = pd.DataFrame(resultados)
print("\n================== RESULTADOS FINALES ==================")
print(df.to_string(index=False))




================== RESULTADOS FINALES ==================
 Tamaño de Matriz  Tiempo Promedio GPU (ms)  Tiempo Promedio CPU (ms)
               10                  34.19126                   0.26788
               30                  30.77398                   3.77488
               50                  28.87600                  13.31368
               70                  30.75938                  34.90046
               90                  33.07840                  78.01806
              110                  41.12534                 161.15190
              130                  48.27518                 261.55400
              150                  42.57560                 416.75880
              170                  56.59502                 562.83810
              190                  72.64700                 757.38408
              210                  86.87508                 899.19676
              230                 102.13194                1068.14258
              250               